<a href="https://colab.research.google.com/github/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/TransferLearning/TransferLearning_Transfer_learning_with_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with MobileNetV2

In this project, transfer learning will be used on a pre-trained CNN to build an Alpaca/Not Alpaca classifier.

<img src="https://github.com/rvraghvender/DeepLearningProjects/blob/main/ConvolutionNeuralNetworks/TransferLearning/images/alpaca.png?raw=true" style="width:300px;height:220px;">

A pre-trained model is a network that's already been trained on a large dataset and saved, which allows us to use it to customize our own model cheaply and efficiently. MobileNetV2, was designed to provide fast and computationally efficient performance. The model is pre-trained on ImageNet, a dataset containing over 14 million images and 1000 classes.